# Machine Learning Engineer Nanodegree
## Capstone Project - Data Preparation
Purpose of this notebook is to access all data sources, gather, clean and prepare all data to be used in capstone project. We will focus on **DMND students** that enrolled in any moment after **2017-04-01** and before **2017-08-10**. We will use as total universe users who are registered and visited DMND NDOP at least once in this period.

## Getting the universe

In [6]:
import pandas as pd
import numpy as np
import udb
import datetime

start_date = datetime.date(year=2017, month=4, day=1)
end_date = datetime.date(year=2017, month=8, day=10)
cut_date = datetime.date(year=2017, month=8, day=10)

br = udb.get_ebdb_engine()
us = udb.get_analytics_engine()

all_accounts_who_visited = pd.read_sql_query("""
SELECT fbi.email, COUNT(fbp.id) AS count
FROM frontend_brazil.pages fbp
  LEFT JOIN frontend_brazil.identifies fbi ON fbp.anonymous_id = fbi.anonymous_id
WHERE fbp.received_at >= '{}' AND fbp.received_at <= '{}'
AND fbp.path LIKE '%%digital-marketing%%'
GROUP BY fbi.email
""".format(start_date.strftime('%Y-%m-%d'), cut_date.strftime('%Y-%m-%d')), con=us)

all_accounts_who_visited.shape[0]

10190

In [8]:
all_accounts_who_visited.drop('count', axis=1, inplace=True)
all_accounts_who_visited.head()

,email
0,falecom@joaoverissimo.com
1,eb.gomes@uol.com.br
2,secatto@terra.com.br
3,leozinho.s.a@hotmail.com
4,mdarlen@gmail.com


Now, let's get who became **paying student** in this period - the target feature we will train our model to predict.

## Paying students

In [9]:
paying_students = pd.read_sql_query("""
SELECT
  au.email
FROM payment_app_subscription ps
  INNER JOIN payment_app_product pp ON ps.product_id = pp.id
  INNER JOIN auth_user au ON au.id = ps.user_id
WHERE (status = 'active' OR status = 'payment_credit_retry')
      AND pp.code like 'nd018%%'
      AND register_date >= '{}'
      AND register_date <= '{}'
""".format(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')), con=br)

paying_students.shape[0]

3599

Next step is to merge both into one dataframe, with **email** and target feature **is_paying_student**, showing 0 or 1.

In [10]:
paying_students['is_paying_student'] = 1
x = pd.merge(all_accounts_who_visited, paying_students, how='outer', on='email')

In [11]:
x['is_paying_student'] = x['is_paying_student'].fillna(0).astype(int)
print(x['is_paying_student'].sum(), x.shape[0])

3599 10965


## How old are these accounts?
Our first hypothesis is that the likelihood of a student to enroll is influenced by how recent they have an account registered at Udacity.

In [12]:
x['email'] = x['email'].astype(str)
email_query = '('
for index, row in x.iterrows():
    if index == x['email'].shape[0] - 1:
        email_query = email_query + "'" + row['email'] + "')"
    else:
        email_query = email_query + "'" + row['email'] + "', "

In [13]:
emails_dates_joined = pd.read_sql_query("SELECT email, date_joined FROM auth_user WHERE email in {}".format(email_query), con=br)
emails_dates_joined.shape[0]

10821

As you can see above, we were not able to find all join dates from the **32,543** accounts in our universe, we found only **31,990**. Let's see if we have a better luck looking into US database:

In [14]:
emails_dates_joined_us = pd.read_sql_query("SELECT email, created_at FROM analytics_tables.accounts WHERE email in {}".format(email_query), con=us)
emails_dates_joined_us.shape[0]

10729

Strangely enough, we found even less data in US database. Let's move forward with Brazil data and disregard rows without join date:

In [15]:
result = pd.merge(x, emails_dates_joined, how='inner', on='email')
result.shape[0]

10821

This is not quite what we want: let's get the age in days, assuming today is the last day of the period:

In [16]:
def calculate_age(row):
    d2 = row['date_joined'].to_pydatetime().date()
    d1 = end_date
    return abs((d2 - d1).days)
    
result['age_in_days'] = result.apply(calculate_age, axis=1)
result.shape[0]

10821

In [17]:
x = pd.merge(x, result[['email', 'age_in_days']], how='inner', on='email')
x.shape[0]

10821

In [18]:
x.head()

,email,is_paying_student,age_in_days
0,falecom@joaoverissimo.com,0,387
1,eb.gomes@uol.com.br,0,294
2,secatto@terra.com.br,0,324
3,leozinho.s.a@hotmail.com,0,343
4,mdarlen@gmail.com,0,222


## How many times has each user enrolled in a webinar?
Doing webinars is one of our key strategies to engage leads. Let's check if it really leads to conversion.

In [19]:
webinar_enrollments = pd.read_sql_query("""
SELECT email, COUNT(id)
FROM brazil_events.event_sign_up
WHERE enrollment_date >= '{}' and enrollment_date <= '{}'
GROUP BY email
""".format(start_date.strftime('%Y-%m-%d'), cut_date.strftime('%Y-%m-%d')), con=us)

webinar_enrollments.shape[0]

23858

In [20]:
x = pd.merge(x, webinar_enrollments, how='left', on='email')
x.shape[0]

10821

In [21]:
x['webinar_enrollments'] = x['count'].fillna(0).astype(int)
x = x[['email', 'is_paying_student', 'age_in_days', 'webinar_enrollments']]
x.head()

,email,is_paying_student,age_in_days,webinar_enrollments
0,falecom@joaoverissimo.com,0,387,0
1,eb.gomes@uol.com.br,0,294,1
2,secatto@terra.com.br,0,324,1
3,leozinho.s.a@hotmail.com,0,343,3
4,mdarlen@gmail.com,0,222,0


## How many times has each user enrolled in a free course?
There are people who believe free courses leads to paying students. Let's see if that's true.

In [24]:
free_course_enrollments = pd.read_sql_query("""
SELECT
  ac.email,
  count(ce.course_key) as course_enrollments
FROM analytics_tables.course_enrollments ce 
INNER JOIN analytics_tables.accounts ac on ce.user_id = ac.user_id
  WHERE ce.join_time <= '{}' AND ac.email IN {}
GROUP BY ac.email
""".format(cut_date.strftime('%Y-%m-%d'), email_query), con=us)

free_course_enrollments.shape[0]

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



OperationalError: (psycopg2.extensions.QueryCanceledError) Query (9296305) canceled on user's request and ran out of wlm queues for restart.
 [SQL: "\nSELECT\n  ac.email,\n  count(ce.course_key) as course_enrollments\nFROM analytics_tables.course_enrollments ce \nINNER JOIN analytics_tables.accounts ac on ce.user_id = ac.user_id\n  WHERE ce.join_time <= '2017-08-10' AND ac.email IN ('falecom@joaoverissimo.com', 'eb.gomes@uol.com.br', 'secatto@terra.com.br', 'leozinho.s.a@hotmail.com', 'mdarlen@gmail.com', 'ae.hirota@gmail.com', 'guta302@hotmail.com', 'brendaholz@gmail.com', 'lwsantos@lwsantos.com.br', 'webs@icons4u.com.br', 'suelenhenderson@gmail.com', 'sandragrohskopfti@gmail.com', 'weinnysoares.wad@gmail.com', 'felipechalreo@gmail.com', 'willian.2a@hotmail.com', 'wesleytaumaturgo@gmail.com', 'jhfcnascimento@gmail.com', 'caique_cp@hotmail.com', 'nelsonmagalhaescouto@gmail.com', 'koelho@gmail.com', 'alinebalarini@hotmail.com', 'foltran012@gmail.com', 'fernandocarvalhodocs@gmail.com', '

In [ ]:
x = pd.merge(x, free_course_enrollments, how='left', on='email')
x.shape[0]

In [ ]:
x['course_enrollments'] = x['course_enrollments'].fillna(0).astype(int)
x.head()

## How many times has each user navigated to each of our key pages?
The navigation pattern should be considered in predicting whether a user will become a paying student or not.

In [ ]:
all_visits = pd.read_sql_query("""
SELECT
  DISTINCT fbp.id,
  fbp.path,
  fbp.referrer,
  fbp.context_user_agent,
  fbi.email
FROM frontend_brazil.pages fbp
  LEFT JOIN frontend_brazil.identifies fbi ON fbi.anonymous_id = fbp.anonymous_id
WHERE fbp.received_at <= '{}' AND fbi.email IN {}
""".format(cut_date.strftime('%Y-%m-%d'), email_query), con=us)

all_visits.shape[0]

In [ ]:
all_visits.head()

In [ ]:
visits = all_visits.copy()

In [ ]:
visits['is_home'] = (visits['path'] == '/').astype(int)
visits['is_ndop'] = (visits['path'].str.contains('--nd')).astype(int)
visits['is_catalog_all'] = (visits['path'] == '/courses/all/').astype(int)
visits['is_catalog_nanodegrees'] = (visits['path'] == '/courses/nanodegrees/').astype(int)
visits['is_nanodegree_home'] = (visits['path'] == '/nanodegree/').astype(int)
visits['is_fcop_ud'] = (visits['path'].str.contains('--ud')).astype(int)
visits['is_fcop_cs'] = (visits['path'].str.contains('--cs')).astype(int)
visits['is_fcop_st'] = (visits['path'].str.contains('--st')).astype(int)
visits['is_signin'] = (visits['path'].str.contains('/signin/')).astype(int)
visits['is_event'] = (visits['path'].str.contains('/events/')).astype(int)
visits['is_50back'] = (visits['path'] == '/nanodegree/50-back/').astype(int)
visits['is_tech_requirements'] = (visits['path'] == '/tech-requirements//').astype(int)
visits['is_contact'] = (visits['path'] == '/contact/').astype(int)
visits['is_us'] = (visits['path'] == '/us/').astype(int)
visits['is_jobs'] = (visits['path'] == '/jobs/').astype(int)
visits['is_legal'] = (visits['path'] == '/legal/').astype(int)
visits['is_hire_talent'] = (visits['path'] == '/hire-talent/').astype(int)
visits['is_business'] = (visits['path'] == '/business/').astype(int)
visits['is_success'] = (visits['path'] == '/success/').astype(int)
visits['is_payment'] = (visits['path'] == '/payment/').astype(int)
visits['is_android'] = (visits['path'].str.contains('/android/')).astype(int)
visits['is_ai'] = (visits['path'].str.contains('/ai/')).astype(int)
visits['is_drive'] = (visits['path'].str.contains('/drive/')).astype(int)
visits['is_robotics'] = (visits['path'].str.contains('/robotics/')).astype(int)
visits['is_checkout'] = (visits['path'].str.contains('/checkout')).astype(int)

In [ ]:
visits.head().transpose()

In [ ]:
def fix_is_ndop(row):
    if row['is_checkout'] == 1:
        return 0
    else:
        return row['is_ndop']

visits['is_ndop'] = visits.apply(fix_is_ndop, axis=1)

In [ ]:
import re

def is_mobile(row):
    if re.search('/Android|webOS|iPhone|iPad|iPod|BlackBerry|IEMobile|Opera Mini/', str(row['context_user_agent'])):
        return 1
    else:
        return 0

visits['is_mobile'] = visits.apply(is_mobile, axis=1)

In [ ]:
visits['referrer'] = visits['referrer'].fillna('')
visits['is_referrer_google'] = (visits['referrer'].str.contains('.google.')).astype(int)
visits['is_referrer_facebook'] = (visits['referrer'].str.contains('.facebook.')).astype(int)
visits['is_referrer_live'] = (visits['referrer'].str.contains('.live.')).astype(int)
visits['is_referrer_infomoney'] = (visits['referrer'].str.contains('.infomoney.')).astype(int)
visits['is_referrer_catracalivre'] = (visits['referrer'].str.contains('.catracalivre.')).astype(int)
visits['is_referrer_android'] = (visits['referrer'].str.contains('.android.')).astype(int)
visits['is_referrer_anhanguera'] = (visits['referrer'].str.contains('anhanguera.')).astype(int)
visits['is_referrer_linkedin'] = (visits['referrer'].str.contains('.linkedin.')).astype(int)
visits['is_referrer_instagram'] = (visits['referrer'].str.contains('.instagram.')).astype(int)
visits['is_referrer_cbsi'] = (visits['referrer'].str.contains('.cbsi.')).astype(int)
visits['is_referrer_tecmundo'] = (visits['referrer'].str.contains('.tecmundo.')).astype(int)
visits['is_referrer_bing'] = (visits['referrer'].str.contains('.bing.')).astype(int)
visits['is_referrer_computerworld'] = (visits['referrer'].str.contains('.computerworld.')).astype(int)
visits['is_referrer_github'] = (visits['referrer'].str.contains('.github.')).astype(int)

In [ ]:
visits.drop('path', axis=1, inplace=True)
visits.drop('referrer', axis=1, inplace=True)
visits.drop('context_user_agent', axis=1, inplace=True)
visits.head().transpose()

Now that we have a dataframe with all visits from our universe of users transformed in features, let's aggregate by email:

In [ ]:
f = {
    'id': ['count'],
    'is_home': ['sum'],
    'is_ndop': ['sum'],
    'is_catalog_all': ['sum'],
    'is_catalog_nanodegrees': ['sum'],
    'is_nanodegree_home': ['sum'],
    'is_fcop_ud': ['sum'],
    'is_fcop_cs': ['sum'],
    'is_fcop_st': ['sum'],
    'is_signin': ['sum'],
    'is_event': ['sum'],
    'is_50back': ['sum'],
    'is_tech_requirements': ['sum'],
    'is_contact': ['sum'],
    'is_us': ['sum'],
    'is_jobs': ['sum'],
    'is_legal': ['sum'],
    'is_hire_talent': ['sum'],
    'is_business': ['sum'],
    'is_success': ['sum'],
    'is_payment': ['sum'],
    'is_android': ['sum'],
    'is_ai': ['sum'],
    'is_drive': ['sum'],
    'is_robotics': ['sum'],
    'is_checkout': ['sum'],
    'is_mobile': ['sum'],
    'is_referrer_google': ['sum'],
    'is_referrer_facebook': ['sum'],
    'is_referrer_live': ['sum'],
    'is_referrer_infomoney': ['sum'],
    'is_referrer_catracalivre': ['sum'],
    'is_referrer_android': ['sum'],
    'is_referrer_anhanguera': ['sum'],
    'is_referrer_linkedin': ['sum'],
    'is_referrer_instagram': ['sum'],
    'is_referrer_cbsi': ['sum'],
    'is_referrer_tecmundo': ['sum'],
    'is_referrer_bing': ['sum'],
    'is_referrer_computerworld': ['sum'],
    'is_referrer_github': ['sum']
}
grouped = visits.groupby('email', as_index=False).agg(f)

In [ ]:
grouped.columns = grouped.columns.droplevel(-1)

In [ ]:
grouped.rename(columns={'id': 'count_visits'}, inplace=True)
grouped.head().transpose()

Now, let's put everything together:

In [ ]:
result = pd.merge(x, grouped, how='inner', on='email')
result.shape

In [ ]:
result.head().transpose()

## How many emails each lead opened?
One of the key strategies we use to nurture leads is email marketing. Let's see if it is working.

In [ ]:
email_data = pd.read_sql_query("""
SELECT context_traits_email AS email, id
FROM vero_email.email_opened 
WHERE received_at <= '{}'
AND context_traits_email IN {}
""".format(cut_date.strftime('%Y-%m-%d'), email_query), con=us)

email_data.shape[0]

In [ ]:
grouped_email_data = email_data.groupby('email')['id'].count()
grouped_email_data.shape[0]

In [ ]:
grouped = pd.DataFrame()
grouped['email'] = grouped_email_data.index.values
grouped['opened_emails'] = grouped_email_data.values
grouped.head()

In [ ]:
final_result = pd.merge(result, grouped, how='left', on='email')
final_result.shape[0]

In [ ]:
final_result['opened_emails'] = final_result['opened_emails'].fillna(0).astype(int)

In [ ]:
final_result.head().transpose()

## Reordering features and saving to a file
Final steps are 1) reordering features, leaving the target **`is_paying_student`** as last feature, and 2) saving into a CSV file:

In [ ]:
for index, value in enumerate(final_result.columns.values):
    print(index, value)

In [ ]:
cols = list(final_result)
cols[46], cols[1] = cols[1], cols[46]
final_data = final_result.ix[:,cols]

In [ ]:
for index, value in enumerate(final_data.columns.values):
    print(index, value)

In [ ]:
final_data.to_csv('data_prepared.csv')